In [ ]:
import numpy as np
import score_matching
import sampling
import updates

In [ ]:
T=5
t=0
dt=0.01
Nt=int((T-t)/dt)

## network size
m=32
nTh=4
d=2

## initial sampling
nex=150**2
sampling_model = sampling.Sampling(nex=nex, d=d, t=t, seed=23)
v = np.array(sampling_model.rejection_sampling())

# initial density
f_net = updates.f_ext(v, t)

fisher_divergence_evo=[]
energy_evo=[]
entropy_decay_rate_evo=[]
traj=[v]

In [ ]:
'''evolution'''
for nt in range(Nt):
    print('current time: ', t)
    model_sm = score_matching.ScoreMatching(v=v, t=t, nex=nex, d=d, m=m, nTh=nTh)
    if nt==0:
        fisher_divergence = model_sm.L2_train(tol=5e-4)
        fisher_divergence_evo.append(fisher_divergence)
    else:
        fisher_divergence = model_sm.ism_train(max_iter=25)
        fisher_divergence_evo.append(fisher_divergence)
    model_sm.save_model()
    
    # compute score and score_jac at t_{n}
    score = model_sm.score(v)
    score_jac = model_sm.score_jac(v)

    model_par = updates.Particle(v=v, score=score, score_jac=score_jac, dt=dt, d=d, C=1/16)
    # compute energy
    energy = updates.energy(v)
    energy_evo.append(energy)
    # compute v at t_{n+1}
    v, entropy_decay_rate = model_par.vel_and_entropy()
    traj.append(v)
    entropy_decay_rate_evo.append(entropy_decay_rate)
    # update density
    f_net = model_par.density(f_net)

    t += dt